In [1]:
import sys
sys.path.append("/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/MIP")
import mip_utils


In [2]:
from z3.z3 import Solver, sat, unsat, Int, Bool, Sum, If, And, Or, Not, Implies, BoolVal, is_true
import time

def solve_mcp_smt(m, n, capacities, sizes, dist_matrix, timeout_sec=300):
    # N = n + 1 (number of points including origin n)
    N = n + 1
    points = list(range(N)) # 0..n
    items = list(range(n)) # 0..n-1
    couriers = list(range(m)) # 0..m-1
    origin = n

    # --- Find Bounds for Binary Search ---
    max_single_dist = 0
    if n > 0:
        max_single_dist = max(dist_matrix[origin][j] + dist_matrix[j][origin] for j in items)

    # A very rough upper bound
    max_possible_dist = sum(max(row) for row in dist_matrix) * n

    lb = max_single_dist
    ub = max_possible_dist # Replace with a better heuristic UB if possible
    
    best_model = None
    optimal_max_dist = ub + 1 # Initialize higher than UB

    start_time = time.time()

    while lb <= ub:
        mid_dist = lb + (ub - lb) // 2
        print(f"Checking MaxDist = {mid_dist} (Bounds: [{lb}, {ub}])")

        s = Solver()
        s.set("timeout", max(1000, int((timeout_sec - (time.time() - start_time)) * 1000))) # Timeout in ms

        # --- Variables ---
        assign = [Int(f"assign_{j}") for j in items]
        use_edge = [[[Bool(f"use_{i}_{p}_{q}") for q in points] for p in points] for i in couriers]
        u = [[Int(f"u_{i}_{p}") for p in points] for i in couriers]
        courier_dist = [Int(f"dist_{i}") for i in couriers]

        # --- Constraints ---

        # C1: Assignment Domain
        for j in items:
            s.add(assign[j] >= 0, assign[j] < m)

        # C2: Capacity
        for i in couriers:
            load = Sum([If(assign[j] == i, sizes[j], 0) for j in items])
            s.add(load <= capacities[i])

        # C3 & C4: Tour Flow and Assignment Linking
        active_couriers = [Bool(f"active_{i}") for i in couriers]
        for i in couriers:
             # Check if courier i is assigned any items
             is_active_cond = Or([assign[j] == i for j in items] if n > 0 else [BoolVal(False)])
             s.add(active_couriers[i] == is_active_cond)
             
             # Edges from/to origin only if active
             s.add(Implies(active_couriers[i], Sum([If(use_edge[i][origin][q], 1, 0) for q in items]) == 1))
             s.add(Implies(active_couriers[i], Sum([If(use_edge[i][p][origin], 1, 0) for p in items]) == 1))
             s.add(Implies(Not(active_couriers[i]), Sum([If(use_edge[i][origin][q], 1, 0) for q in items]) == 0))
             s.add(Implies(Not(active_couriers[i]), Sum([If(use_edge[i][p][origin], 1, 0) for p in items]) == 0))

             for j in items:
                 # In-degree for item j by courier i
                 in_degree_j = Sum([If(use_edge[i][p][j], 1, 0) for p in points])
                 s.add(Implies(assign[j] == i, in_degree_j == 1))
                 s.add(Implies(assign[j] != i, in_degree_j == 0))

                 # Out-degree for item j by courier i
                 out_degree_j = Sum([If(use_edge[i][j][q], 1, 0) for q in points])
                 s.add(Implies(assign[j] == i, out_degree_j == 1))
                 s.add(Implies(assign[j] != i, out_degree_j == 0))
             
             # Ensure no edges used if inactive (stricter version)
             s.add(Implies(Not(active_couriers[i]), And([Not(use_edge[i][p][q]) for p in points for q in points])))


        # C5: No Self-Loops
        for i in couriers:
            for p in points:
                s.add(Not(use_edge[i][p][p]))

        # C6: Sub-tour Elimination (MTZ)
        for i in couriers:
            s.add(u[i][origin] == 0)
            for j in items:
                 s.add(Implies(assign[j] == i, u[i][j] >= 1))
                 # Simple upper bound, could potentially be tightened
                 s.add(Implies(assign[j] == i, u[i][j] <= n)) 
                 # Ensure u[j] is only constrained if assigned to i
                 s.add(Implies(assign[j] != i, u[i][j] == 0)) # Or some other default

            for p in points:
                for q in items: # Only need q != origin for MTZ constraint
                    s.add(Implies(use_edge[i][p][q], u[i][q] >= u[i][p] + 1))

        # C7: Calculate Courier Distance
        for i in couriers:
            dist_expr = Sum([If(use_edge[i][p][q], dist_matrix[p][q], 0)
                             for p in points for q in points])
            s.add(courier_dist[i] == dist_expr)

        # C8: Objective Constraint
        for i in couriers:
            s.add(courier_dist[i] <= mid_dist)

        # --- Check Satisfiability ---
        result = s.check()
        elapsed_time = time.time() - start_time

        if result == sat:
            print(f"  SAT (Found solution with MaxDist <= {mid_dist})")
            optimal_max_dist = mid_dist
            best_model = s.model()
            ub = mid_dist - 1 # Try for better
        elif result == unsat:
            print(f"  UNSAT (No solution with MaxDist <= {mid_dist})")
            lb = mid_dist + 1 # Need larger distance
        else: # unknown (timeout)
            print(f"  UNKNOWN (Timeout at MaxDist = {mid_dist})")
            # If we timeout, we can't be sure if mid_dist is feasible or not.
            # A common strategy is to stop improving UB here, or maybe increase LB.
            # Let's increase LB to avoid getting stuck.
            lb = mid_dist + 1 
            # If time is up completely, break the loop
            if elapsed_time >= timeout_sec:
                 print("Timeout reached during binary search.")
                 break
                 
        # Check overall timeout
        if elapsed_time >= timeout_sec:
            print("Timeout reached during binary search.")
            break


    # --- Process Result ---
    total_time = time.time() - start_time
    is_optimal = (best_model is not None) and (lb > ub) # Optimal if search finished normally

    if best_model:
        # Extract solution
        final_assign = {j: best_model.eval(assign[j]).as_long() for j in items}
        final_tours = [[] for _ in couriers]
        final_obj = 0

        for i in couriers:
            # Check if courier is active by seeing if any item is assigned
            if any(final_assign.get(j, -1) == i for j in items):
                current = origin
                tour_i = []
                visited_count = 0 # Safety break
                while visited_count <= n: 
                    found_next = False
                    for q in points:
                        use = best_model.eval(use_edge[i][current][q])
                        if is_true(use):
                            if q == origin: # Back to origin
                                found_next = True 
                                break 
                            tour_i.append(q) # q is an item index
                            current = q
                            found_next = True
                            break
                    visited_count += 1
                    if current == origin or not found_next: # Finished tour or error
                         break
                final_tours[i] = tour_i
            
            # Get objective value (max distance found)
            dist_val = best_model.eval(courier_dist[i]).as_long()
            final_obj = max(final_obj, dist_val)
            

        return {
            "time": int(total_time),
            "optimal": is_optimal,
            "obj": final_obj if final_obj <= optimal_max_dist else optimal_max_dist, # Report the confirmed feasible objective
            "sol": final_tours
        }
    else:
        # No solution found within bounds or time limit
         return {
            "time": int(total_time),
            "optimal": False,
            "obj": -1, # Or some indicator of no solution found
            "sol": [] 
        }

# Example Usage (replace with actual data loading)
# m, n, capacities, sizes, dist_matrix = load_instance("instance.txt")
# result = solve_mcp_smt(m, n, capacities, sizes, dist_matrix)
# print(result) 
# save_to_json(result, "res/SMT/instance_result.json")

In [7]:
inst01

{'m': 6,
 'n': 17,
 'capacities': {0: 190, 1: 185, 2: 185, 3: 190, 4: 195, 5: 185},
 'sizes': {0: 11,
  1: 11,
  2: 23,
  3: 16,
  4: 2,
  5: 1,
  6: 24,
  7: 14,
  8: 20,
  9: 10,
  10: 11,
  11: 22,
  12: 2,
  13: 12,
  14: 9,
  15: 21,
  16: 10},
 'distances': {(0, 0): 0,
  (0, 1): 20,
  (0, 2): 19,
  (0, 3): 28,
  (0, 4): 58,
  (0, 5): 48,
  (0, 6): 45,
  (0, 7): 32,
  (0, 8): 90,
  (0, 9): 61,
  (0, 10): 71,
  (0, 11): 59,
  (0, 12): 65,
  (0, 13): 46,
  (0, 14): 72,
  (0, 15): 51,
  (0, 16): 46,
  (0, 17): 66,
  (1, 0): 103,
  (1, 1): 0,
  (1, 2): 81,
  (1, 3): 107,
  (1, 4): 38,
  (1, 5): 110,
  (1, 6): 55,
  (1, 7): 94,
  (1, 8): 76,
  (1, 9): 123,
  (1, 10): 88,
  (1, 11): 76,
  (1, 12): 69,
  (1, 13): 86,
  (1, 14): 99,
  (1, 15): 113,
  (1, 16): 108,
  (1, 17): 106,
  (2, 0): 73,
  (2, 1): 41,
  (2, 2): 0,
  (2, 3): 80,
  (2, 4): 40,
  (2, 5): 29,
  (2, 6): 26,
  (2, 7): 13,
  (2, 8): 75,
  (2, 9): 42,
  (2, 10): 70,
  (2, 11): 58,
  (2, 12): 46,
  (2, 13): 27,
  (2, 14): 53

In [ ]:
inst01 = mip_utils.parse_instance("/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/instances/inst07.dat")
# # Convert distances dictionary with tuple keys to a 2D list
# n_plus_1 = inst01["n"] + 1  # Include origin
# dist_matrix = [[0] * n_plus_1 for _ in range(n_plus_1)]
# for (i, j), dist in inst01["distances"].items():
# 	dist_matrix[i][j] = dist

# Call the function with the converted distance matrix
result = solve_mcp_smt(inst01["m"], inst01["n"], inst01["capacities"], inst01["sizes"], inst01["distances"])
print(result)

KeyError: 17

In [1]:
import model_smt
import sys
sys.path.append("/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/MIP")
import mip_utils

inst01 = mip_utils.parse_instance("/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/instances/inst07.dat")
result1 = model_smt.solve_mcp_smt(
    inst01["m"],
    inst01["n"],
    inst01["capacities"],
    inst01["sizes"],
    inst01["distances"],
    timeout_sec=300,
    use_symmetry_breaking=False
)

Initial Bounds: LB=167, UB=2091
Checking MaxDist = 1129 (Bounds: [167, 2091]) Symmetry: False
  SAT (Found solution with MaxDist <= 1129)
Checking MaxDist = 647 (Bounds: [167, 1128]) Symmetry: False
  SAT (Found solution with MaxDist <= 647)
Checking MaxDist = 406 (Bounds: [167, 646]) Symmetry: False
  SAT (Found solution with MaxDist <= 406)
Checking MaxDist = 286 (Bounds: [167, 405]) Symmetry: False
  SAT (Found solution with MaxDist <= 286)
Checking MaxDist = 226 (Bounds: [167, 285]) Symmetry: False
  SAT (Found solution with MaxDist <= 226)
Checking MaxDist = 196 (Bounds: [167, 225]) Symmetry: False
  SAT (Found solution with MaxDist <= 196)
Checking MaxDist = 181 (Bounds: [167, 195]) Symmetry: False
  SAT (Found solution with MaxDist <= 181)
Checking MaxDist = 173 (Bounds: [167, 180]) Symmetry: False
  SAT (Found solution with MaxDist <= 173)
Checking MaxDist = 169 (Bounds: [167, 172]) Symmetry: False
  SAT (Found solution with MaxDist <= 169)
Checking MaxDist = 167 (Bounds: [167,

In [2]:
print(result1)

{'time': 27, 'optimal': True, 'obj': 167, 'sol': [[16, 1], [14, 9, 15, 5, 7], [10, 2, 6], [8, 13], [12, 0, 3], [4, 11]]}


In [31]:
mip_utils.write_output(result1, "solutions/inst07.json")

Results written to solutions/inst07.json


In [36]:
! python solution_checker.py "/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/instances" "solutions/"


Checking results in SMT folder

Checking results in solutions/SMT folder
	Checking results for instance inst02.json
	Loading input instance /Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/instances/inst02.dat
		Checking solver mip_pulp
Traceback (most recent call last):
  File "/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/SMT/solution_checker.py", line 129, in <module>
    main(sys.argv)
  File "/Users/giacomopiergentili/Developer/CDMO/project-final/CDMO-project/SMT/solution_checker.py", line 93, in main
    dist += dist_matrix[prev_item][curr_item]
            ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
IndexError: list index out of range
